In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seysmo.models.model_class import CNN, CNNLSTMNetwork, CNNLSTMNetwork2
from seysmo.models.train_model import train, evaluate
from seysmo.models.utils import save_model, load_model, count_parameters, EarlyStopper, SignalSpeedDataset, give_data
from seysmo.visualization.plotting import plot_map
from seysmo.features.mapping import do_array_for_mapping, compute_y_pred
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import MeanAbsolutePercentageError

import mlflow

mlflow.set_tracking_uri("http://localhost:5000")

In [2]:
X_train, y_train, coord_train, X_val, y_val, coord_val, X_test, y_test, coord_test = give_data(
    '../data/processed/coord_dict.pkl')

In [3]:
X_train.shape

(11045, 33, 438)

In [19]:
X_train = np.transpose(X_train, (0,2,1))
X_train.shape

(11045, 438, 33)

In [20]:
X_val = np.transpose(X_val, (0,2,1))
X_test = np.transpose(X_test, (0,2,1))

In [21]:
from torch.utils.data import Dataset


class SignalSpeedDataset(Dataset):
    def __init__(self, X, y):
        # Добавляем размер канала
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [22]:
train_dataset = SignalSpeedDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = SignalSpeedDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_dataset = SignalSpeedDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [23]:
train_dataset.X.shape

torch.Size([11045, 438, 33])

In [24]:
for batch, (X, y) in enumerate(train_dataloader):
    print(X.shape)
    break

torch.Size([32, 438, 33])


In [30]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Параметры модели
input_size = 33
hidden_size = 128
num_layers = 50
output_size = 10

# Создание модели
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to('cuda')

# Определение функции потерь и оптимизатора
loss_fn = nn.MSELoss()
metric_fn = MeanAbsolutePercentageError().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for t in range(100):
        print(f"Epoch {t + 1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, metric_fn, optimizer, epoch=t, device='cuda',
              output_size=output_size)
        validation_loss, validation_mape = evaluate(val_dataloader, model, loss_fn, metric_fn, epoch=t, device='cuda',
                                                    output_size=output_size)

Epoch 1
-------------------------------
loss: 128584.398438 MAPE: 0.999889 [0 / 346]
loss: 118726.429688 MAPE: 0.950824 [100 / 346]
loss: 101167.101562 MAPE: 0.904233 [200 / 346]
loss: 102556.828125 MAPE: 0.866204 [300 / 346]
Eval metrics: 
MAPE: 0.85, Avg loss: 97603.413836 

Epoch 2
-------------------------------
loss: 89223.570312 MAPE: 0.840632 [0 / 346]
loss: 89540.671875 MAPE: 0.803572 [100 / 346]
loss: 84388.156250 MAPE: 0.770642 [200 / 346]
loss: 72825.828125 MAPE: 0.726237 [300 / 346]
Eval metrics: 
MAPE: 0.71, Avg loss: 75286.518569 

Epoch 3
-------------------------------
loss: 71713.468750 MAPE: 0.709065 [0 / 346]
loss: 70770.914062 MAPE: 0.679115 [100 / 346]
loss: 63913.976562 MAPE: 0.637495 [200 / 346]
loss: 58504.355469 MAPE: 0.607172 [300 / 346]
Eval metrics: 
MAPE: 0.59, Avg loss: 57510.007360 

Epoch 4
-------------------------------
loss: 56954.039062 MAPE: 0.593504 [0 / 346]
loss: 51283.839844 MAPE: 0.551282 [100 / 346]
loss: 47130.476562 MAPE: 0.515089 [200 / 346